In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import binarize
import sklearn
from sklearn.svm import LinearSVC
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import nltk
import os
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

In [4]:
columns=['value','twitterhandle','userid','user_screen_name','retweets','favorites','tweet_id','timestamp','tweet']
#df=pd.read_csv('/home/ds/notebooks/Data/twitterTweets.csv',sep='\t',names=columns,encoding='utf-8')

In [2]:
def read_csv(name):
    location='/home/ds/notebooks/Data/'
    name=name+'.csv'
    columns=['value','twitterhandle','userid','user_screen_name','retweets','favorites','tweet_id','timestamp','tweet']
    for root,dirs,files in os.walk(location):
        if name in files:
            outfile=os.path.join(root,name)
            return pd.read_csv(outfile,sep='\t',names=columns,encoding='utf-8')#,usecols=['value', 'tweet'])
            
def training(df):
    return np.random.rand(len(df)) < 0.8
#df_train=df[msk]
#df_test=df[~msk]
#return df_train.shape,df_test.shape
#def 

In [3]:
df = read_csv('twitterTweets')

/opt/ds/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
df_train=df[training(df)]
df_test=df[~training(df)]

In [5]:
df2=pd.read_csv('/home/ds/notebooks/Data/twitterhandles.csv',encoding='utf-8')
df2.rename(columns={'Handle': 'value'}, inplace=True)
df_train=pd.merge(df_train, df2, on='value', how='inner')
df_test=pd.merge(df_test, df2, on='value', how='inner')
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
df_train['age_group'] = pd.cut(df_train.Age, range(0, 81, 10), right=False, labels=labels)
df_test['age_group'] = pd.cut(df_test.Age, range(0, 81, 10), right=False, labels=labels)
df_train=df_train.dropna()
df_test=df_test.dropna()

In [6]:
%timeit 
df3.isnull().sum()

NameError: name 'df3' is not defined

In [6]:
def vectorize(df):
    bow_transformer = CountVectorizer(tokenizer=nltk.word_tokenize,token_pattern="[a-zA-Z]{2,15}",stop_words="english")
    bow_transformer.fit(df)
    messages_bow = bow_transformer.transform(df)
    tfidf_transformer = TfidfTransformer().fit(messages_bow)
    messages_tfidf = tfidf_transformer.transform(messages_bow)
    return messages_tfidf    

In [16]:
age_inference = MultinomialNB().fit(vectorize(df_train['tweet']), df_train['age_group'])

In [8]:
bow_transformer = CountVectorizer(tokenizer=nltk.word_tokenize,token_pattern="[a-zA-Z]{2,15}",stop_words="english"
                                 ).fit(df_train['tweet'])

In [9]:
message3 = df_train['tweet'][3]
print message3
bow4 = bow_transformer.transform([message3])
print bow4
print bow4.shape
print bow_transformer.get_feature_names()[129168]
print bow_transformer.get_feature_names()[115477]

@JaffeKaylahttps://twitter.com/thediyideas/status/723936823022473217 …
  (0, 19449)	1
  (0, 29876)	1
  (0, 29910)	1
  (0, 71535)	1
  (0, 133180)	1
(1, 135062)
witnesses
stephanie5g6h


In [10]:
messages_bow = bow_transformer.transform(df_train['tweet'])
print 'sparse matrix shape:', messages_bow.shape
print 'number of non-zeros:', messages_bow.nnz
print 'sparsity: %.2f%%' % (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1]))

sparse matrix shape: (181717, 135062)
number of non-zeros: 1485790
sparsity: 0.01%


In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf4 = tfidf_transformer.transform(bow4)
print tfidf4

  (0, 133180)	0.202814424478
  (0, 71535)	0.662868454503
  (0, 29910)	0.0845306454156
  (0, 29876)	0.139425992904
  (0, 19449)	0.70205888908


In [13]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print messages_tfidf.shape

(181717, 135062)


In [8]:
from sklearn.naive_bayes import MultinomialNB
#%time age_inference = MultinomialNB().fit(messages_tfidf, df_train['age_group'])

In [17]:
print 'predicted:', age_inference.predict(tfidf4)[:1]
print 'expected:', df_train['age_group'][:1]

predicted: ['10-19']
expected: 0    10-19
Name: age_group, dtype: category
Categories (8, object): [0-9 < 10-19 < 20-29 < 30-39 < 40-49 < 50-59 < 60-69 < 70-79]


In [18]:
all_predictions = age_inference.predict(messages_tfidf)
print all_predictions[:10]

['10-19' '10-19' '10-19' '10-19' '10-19' '10-19' '10-19' '10-19' '10-19'
 '10-19']


In [19]:
pd.DataFrame(all_predictions).groupby(0).size()

0
0-9         811
10-19    135360
20-29     10963
30-39      8137
40-49     25072
50-59      1059
60-69       310
70-79         5
dtype: int64

In [20]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score,confusion_matrix

In [21]:
print 'accuracy', accuracy_score(df_train['age_group'], all_predictions)
print 'confusion matrix\n', confusion_matrix(df_train['age_group'], all_predictions)
print '(row=expected, col=predicted)'

accuracy 0.818178816511
confusion matrix
[[   811    554      7      1     59      0      0      0]
 [     0 103385      2      0     48      0      0      0]
 [     0  10025  10931      2     72      0      0      0]
 [     0   7497      1   8088    327      0      0      0]
 [     0   7246      3      3  24088      0      0      0]
 [     0   3661     14     13    202   1059      0      0]
 [     0   2427      4     22    213      0    310      0]
 [     0    565      1      8     63      0      0      5]]
(row=expected, col=predicted)


In [22]:
confusion=pd.DataFrame(confusion_matrix(df_train['age_group'], all_predictions))
confusion.index = ["%s (actual)" % x for x in labels]
confusion.columns = ["%s (predicted)" % x for x in labels] 
confusion

,0-9 (predicted),10-19 (predicted),20-29 (predicted),30-39 (predicted),40-49 (predicted),50-59 (predicted),60-69 (predicted),70-79 (predicted)
0-9 (actual),811,554,7,1,59,0,0,0
10-19 (actual),0,103385,2,0,48,0,0,0
20-29 (actual),0,10025,10931,2,72,0,0,0
30-39 (actual),0,7497,1,8088,327,0,0,0
40-49 (actual),0,7246,3,3,24088,0,0,0
50-59 (actual),0,3661,14,13,202,1059,0,0
60-69 (actual),0,2427,4,22,213,0,310,0
70-79 (actual),0,565,1,8,63,0,0,5


In [23]:
messages_bow_test=bow_transformer.transform(df_test['tweet'])
messages_tfidf_test=tfidf_transformer.transform(messages_bow_test)

In [24]:
all_predictions_test=age_inference.predict(messages_tfidf_test)

In [25]:
print 'accuracy', accuracy_score(df_test['age_group'], all_predictions_test)
print 'confusion matrix\n', confusion_matrix(df_test['age_group'], all_predictions_test)
#print '(row=expected, col=predicted)'

accuracy 0.81241231147
confusion matrix
[[  216   133     0     0     8     0     0     0]
 [    0 26025     1     0    21     0     0     0]
 [    0  2603  2643     0    23     0     0     0]
 [    0  1954     1  1943    81     0     0     0]
 [    0  1901     2     0  5874     0     0     0]
 [    0   953     2     6    46   277     0     0]
 [    0   596     1     1    50     0    80     0]
 [    0   163     0     0    11     0     0     1]]


In [26]:
confusion=pd.DataFrame(confusion_matrix(df_test['age_group'], all_predictions_test))
confusion.index = ["%s (actual)" % x for x in labels]
confusion.columns = ["%s (predicted)" % x for x in labels] 
confusion

,0-9 (predicted),10-19 (predicted),20-29 (predicted),30-39 (predicted),40-49 (predicted),50-59 (predicted),60-69 (predicted),70-79 (predicted)
0-9 (actual),216,133,0,0,8,0,0,0
10-19 (actual),0,26025,1,0,21,0,0,0
20-29 (actual),0,2603,2643,0,23,0,0,0
30-39 (actual),0,1954,1,1943,81,0,0,0
40-49 (actual),0,1901,2,0,5874,0,0,0
50-59 (actual),0,953,2,6,46,277,0,0
60-69 (actual),0,596,1,1,50,0,80,0
70-79 (actual),0,163,0,0,11,0,0,1


In [27]:
pd.DataFrame(all_predictions_test).groupby(0).size()

0
0-9        216
10-19    34328
20-29     2650
30-39     1950
40-49     6114
50-59      277
60-69       80
70-79        1
dtype: int64

In [ ]:
rf = RandomForestRegressor()
rf.fit(X, Y)
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True)

In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
df_train['Age_Encoded']=LabelEncoder().fit_transform(df_train['age_group']) 
#rf = RandomForestRegressor()
#RF=rf.fit(messages_tfidf, df_train['Age_Encoded'])

In [29]:
df_train.groupby('Age_Encoded').size()

Age_Encoded
0      1432
1    103435
2     21030
3     15913
4     31340
5      4949
6      2976
7       642
dtype: int64

In [ ]:
%time age_inference_features = RandomForestRegressor().fit(messages_tfidf, df_train['Age_Encoded'])

CPU times: user 43min 30s, sys: 192 ms, total: 43min 30s
Wall time: 43min 36s


In [ ]:
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), age_inference_features.feature_importances_), names), 
             reverse=True)

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=2)
random_forest=rf.fit(messages_tfidf,df_train['age_group'])
#.predict(messages_tfidf_test)

In [31]:
confusion=pd.DataFrame(confusion_matrix(df_test['age_group'], random_forest.predict(messages_tfidf_test)))
confusion.index = ["%s (actual)" % x for x in labels]
confusion.columns = ["%s (predicted)" % x for x in labels] 
confusion

,0-9 (predicted),10-19 (predicted),20-29 (predicted),30-39 (predicted),40-49 (predicted),50-59 (predicted),60-69 (predicted),70-79 (predicted)
0-9 (actual),357,0,0,0,0,0,0,0
10-19 (actual),1,25969,21,20,36,0,0,0
20-29 (actual),0,551,4701,5,12,0,0,0
30-39 (actual),0,217,8,3710,42,1,1,0
40-49 (actual),1,414,10,18,7326,1,6,1
50-59 (actual),0,120,7,8,17,1131,1,0
60-69 (actual),0,94,1,4,9,2,618,0
70-79 (actual),0,17,2,3,7,1,0,145


In [32]:
print 'accuracy', accuracy_score(df_test['age_group'],random_forest.predict(messages_tfidf_test))

accuracy 0.963631182041


In [54]:
x='I am now 25 years old and my eyesight is worsening, at what point do I get adult supervision? :nerd_face:'

In [60]:
import codecs
fo = codecs.open('/home/ds/notebooks/Data/tweet_test.txt', "r", "utf-8")
lines = fo.readlines()
fo.close()

In [67]:
for x in lines:
    tweet_test= bow_transformer.transform(x.strip())

In [70]:
test=age_inference.predict(tweet_test)

In [86]:
print 'tweet', lines[]
print 'predicted:', random_forest.predict(tweet_test)[3]

tweet My mom needs to chill.

predicted: 10-19


In [ ]:
for x in tweet_test:
    random_forest.predict(x)

In [91]:
pd.DataFrame(random_forest.predict(tweet_test)).groupby(0).size()

0
10-19    130
50-59      6
dtype: int64

In [96]:
pd.DataFrame(zip(lines,random_forest.predict(tweet_test))).to_csv('/home/ds/notebooks/Data/output.csv',encoding='utf8')

In [97]:
df_train.shape

(181758, 12)

In [53]:
#df.groupby(['name','month'])['text'].apply(lambda x: ','.join(x)).reset_index()
df.groupby(['userid'])['tweet'].apply(lambda x: ''.join(str(x))).reset_index().head()

,userid,tweet
0,000Judge1,257232 @BinyominS he is arrived no one is l...
1,0017Daniel,246707 @MrDfferent @M...
2,007Paulami,242644 @SrBachchan intriguing trailer...\nN...
3,007Ranjan,243262 TE3N Official Trailer | Releases 10t...
4,007Rex_Inc,178646 Sweet! A cup from @DiamondGirl127 th...


In [57]:
df.groupby(['userid']).agg({'tweet':'count'}).head()

,tweet
userid,
000Judge1,3
0017Daniel,4
007Paulami,1
007Ranjan,1
007Rex_Inc,1


In [95]:
df_1=df.groupby(['value','Age','Gender','age_group'])['tweet'].apply(lambda x: ''.join(str(x))).reset_index()
df_1.head()

KeyError: 'Age'

In [62]:
df_test2=df_test.groupby(['value','Age','Gender','age_group'])['tweet'].apply(lambda x: ''.join(str(x))).reset_index()

In [88]:
age_inference_2 = MultinomialNB().fit(vectorize(df_train2['tweet']), df_train2['age_group'])

In [86]:
bow_transformer = CountVectorizer(tokenizer=nltk.word_tokenize,token_pattern="[a-zA-Z]{2,15}",stop_words="english"
                                 ).fit(df_train2['tweet'])

In [89]:
messages_bow_test_2=bow_transformer.transform(df_test2['tweet'])
messages_tfidf_test_2=tfidf_transformer.transform(messages_bow_test_2)
all_predictions_test_2=age_inference_2.predict(messages_tfidf_test_2)

ValueError: Input has n_features=17045 while the model has been trained with n_features=135062

In [68]:
confusion=pd.DataFrame(confusion_matrix(df_test2['age_group'], all_predictions_test_2))
confusion.index = ["%s (actual)" % x for x in labels]
confusion.columns = ["%s (predicted)" % x for x in labels] 
confusion

ValueError: Found arrays with inconsistent numbers of samples: [  172 45616]

In [94]:
pd.DataFrame(all_predictions_test_2).groupby(0).size()#.reshape(45616,1)

0
0-9        216
10-19    34328
20-29     2650
30-39     1950
40-49     6114
50-59      277
60-69       80
70-79        1
dtype: int64

In [84]:
messages_tfidf_test_2.shape

(172, 135062)